# Module 3 - Statistics with python
---------------------------------------------------

# Table of Content <a id='toc'></a>

<br>

1. **[SciPy.stats and statistics in python](#stats)**  
    1.1 [manipulation of random distributions](#stats.1)  
     1.1.1 [Drawing some random numbers : rvs](#stats.1.1)  
     1.1.2 [Looking up the quantiles and probability density functions](#stats.1.2)  
    1.2 [t-test](#stats.2)  
    1.3 [statistical power calculations](#stats.3)  
    1.4 [Multiple hypothesis testing](#stats.4)  
    1.5 [Fisher's exact test and the Chi-square test](#stats.5)  
    1.6 [1-way anova](#stats.6)  
    <br>

2. **[Correlation and linear regression](#reg)**  
    2.1 [Correlation](#reg.1)  
    2.2 [Regression](#reg.2)  
    <br>

<br>

## Introduction

This notebook aims to present how to perform classical statistical procedure as well as some amount of regression using various python libraries, such as **`scipy`**.

It **does not aim to replace a course on statistics**, but rather focuses on the code aspect.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import numpy as np
import statsmodels
from IPython.display import Image

print("All libraries imported successfully!")

> Note: if you are missing some of the above modules, you should install them.
>
>    * Installation with **pip**: `pip install --user matplotlib seaborn scipy pandas numpy statsmodels` 
>    * Installation with **conda**: `conda install -c conda-forge matplotlib seaborn scipy pandas numpy statsmodels`

<br>

Making the plotted labels a bit bigger for presentation with a projector... you do **not need to run this cell**.

In [ ]:
import matplotlib as mpl

font = {
    "family": "DejaVu Sans",
    "weight": "bold",
    "size"  : 20
}
mpl.rc("font", **font)

<br>
<br>

# 1. `scipy.stats` and statistics in python <a id='stats'></a>
-----------------------------------------------------------

**[SciPy](https://scipy.org)** is a comprehensive project for scientific python programming, regrouping a [library](https://docs.scipy.org/doc/scipy/reference/) and implementing various tools and algorithm for scientific computation.

This section gives a primer on the **`scipy.stats`** library, which provides ways to interact with various random distribution functions, and implements numerous statistical tests.

<br>


## 1.1 manipulation of random distributions <a id='stats.1'></a>

The **`scipy.stats`** module implements utilities for a large number of continuous and discrete distributions:

In [ ]:
from scipy import stats

dist_continu = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_continuous)]
dist_discrete = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_discrete)]
print("Number of continuous distributions: %d" % len(dist_continu))
print("Number of discrete distributions:   %d" % len(dist_discrete))


Let's experiment with the normal distribution, or **`norm`** in `scipy.stats`

A look at `help(stats.norm)` tells us that:
* The **`loc`** argument (location) specifies the **mean** of the distribution.
* The **`scale`** argument specifies the **standard deviation** of the distribution.

<br>

**Example:** generate a specific normal distribution with a mean of 10 and a stdev of 2.

In [ ]:
N = stats.norm(loc=10, scale=2)

# The mean and variance of a distribution can be retrieved using the .stats method:
print("Type of N is:", type(N))
print("Mean and variance of distribution:", N.stats())


In [ ]:
help(N.stats)

The scipy distribution object (`N` in our example) can then be used to interact with the distribution in many ways, as illustrated below.

<br>

### 1.1.1. Drawing random numbers: `rvs` <a id='stats.1.1'></a>

The **`rvs()`** method allows to draw a random number of values from a distribution.
* The `size` argument can be an integer or a sequence of several integers, this defines the
  dimensions of the returned arrays of random numbers.

In [ ]:
# Get a sample of 10 random numbers, returned as a 1-dimensional array.
N.rvs(size=10)

In [ ]:
# Get a sample of 10 random numbers, returned as a 2-dimensional array (5 rows, 2 cols).
N.rvs(size=(5, 2)) 

In [ ]:
# Draw 5000 random numbers from the distribution and plot them.
plt.hist(N.rvs(size=5000)) 

<div class="alert alert-block alert-warning">
    
**Warning:** as with any drawing of random variable on a computer, [one merely emulates randomness](https://en.wikipedia.org/wiki/Pseudorandom_number_generator).  

<div>

The positive aspect of using pseudo-random numbers is that one can make random operation reproducible by setting up **random seed**.

In [ ]:
import numpy as np

# Set the random seed and draw 5 random numbers:
np.random.seed(2023)   
draw1 = N.rvs(size=5)

# Set the random seed back to the same value as above -> 2021.
np.random.seed(2023)
draw2 = N.rvs(size=5)

print("Are the random draws equal?", all(draw1 == draw2))
print(draw1)
print(draw2)

<br>

### 1.1.2 Looking-up quantiles and probability density functions <a id='stats.1.2'></a>

* **`pdf()`: Probability Density Function** - the probability that a random number has a value equal
  (or close to) the value passed to the function.
* **`cdf()`: Cumulative Distribution Function** - the probability that a random number has a value lower or 
  equal to the value passed to the function.
* **`ppf()`: Percent Point Function** (inverse of CDF) - gives the quantiles of the distribution.

<br>

**Example:** using the PDF to plot a distribution.

In [ ]:
import matplotlib.pyplot as plt 

X = np.arange(0, 20, 0.1)      # Generate a sequence ranging from 0 to 20 with increment of 0.1
plt.plot(X, N.pdf(X))          # For each element of the sequence, get the associated PDF value.
plt.show()

<br>

**Example:** using the CDF to estimate a probability, using PPF to compute quantiles.

In [ ]:
# cdf: Cumulative Distribution Function.
print('What is the probability of drawing a number <= 15.0 ?', N.cdf(15.0))

# ppf: gives the quantiles of the distribution.
P = [0.025, 0.5, 0.975]
Q = N.ppf(P)
print('Quantiles:', P, '->', Q)

<br>

For **discrete distribution** these rules change a bit: `pdf()` is replaced by `pmf()`.

Let's see an example of **binomial distribution** with 10 draws and a 0.5 probability of success:

In [ ]:
X = np.arange(0,10)   # Generate a sequence of integer number from 0 to 9.
plt.scatter(X, stats.binom.pmf(X, n=10, p=0.5))
plt.show()

Most distributions have a certain number of **parameters** which control their overall **shape, location or scale**.
* The **normal distribution** e.g. has two parameters: its mean ($\mu$) and its standard deviation ($\sigma$), 
  which respectively control its location and scale. In `scipy`, these are set via the `loc` (location)
  and `scale` arguments.

In [ ]:
# Generate 100 equally separated points between -5 and 5,
x = np.linspace(-10, 10, 100)

fig, ax = plt.subplots(figsize=(14, 7))

a = sns.lineplot(
    x=x, y=stats.norm.pdf(x, loc=0 , scale=1),
    ax=ax, label="mean=0, stdev=1"
)
sns.lineplot(
    x=x, y=stats.norm.pdf(x, loc=-2, scale=1) ,
    ax=ax, label="mean=-2, stdev=1"
)
sns.lineplot(
    x=x, y=stats.norm.pdf(x, loc=1, scale=3) ,
    ax=ax, label="mean=1, stdev=3")

a.set(xlabel="value", ylabel="density")
a.set_title("normal law - probability density function")
ax.legend()

<br>
<br>

[Back to ToC](#toc)

## 1.2 t-test <a id='stats.2'></a>

[**Student's t-test**](https://en.wikipedia.org/wiki/Student%27s_t-test) is used to determine if the means of two samples (drawn from 2 sub-populations for instance) are significantly different.

It is a widely used test, with important but not overly complex assumptions:
 * Independence of data points.
 * The means of each sample should follow normal distributions.
 * *The two sample share the same variance* (there are different flavors of the t-test depending
   on that assumption).

<br>

**Example:** weight of mice of different genotypes and subjected to different diets.

In [ ]:
mice_data = pd.read_csv("data/mice_data.csv")
sns.catplot(x="weight", y="genotype", data=mice_data, kind="violin")

The t-test has a number of assumptions, mostly the normality of means of each group.

We can get an approximate idea of this with a pair of QQplots.

In [ ]:
WT_data = mice_data["weight"][mice_data["genotype"] == "WT"]
KO_data = mice_data["weight"][mice_data["genotype"] == "KO"]

fig,ax = plt.subplots(1, 2, figsize=(14, 7))
stats.probplot(WT_data, dist="norm", plot=ax[0], rvalue=True)
ax[0].set_title("WT data")
stats.probplot(KO_data, dist="norm", plot=ax[1], rvalue=True)
ax[1].set_title("KO data")
fig.tight_layout()

And now for the t-test itself:

In [ ]:
# Note: use equal_var=True if you have tested for variance equality.
tstat, pval = stats.ttest_ind(KO_data, WT_data, equal_var=False)

print("test statistic:", tstat)
print("p-value       :", pval)

<br>

<div class="alert alert-block alert-success">

### Micro-Exercise 1
* There is another column in the `mice_data` data-set : `diet`, which can take the values `"HFD"` and `"CHOW"`.
  Perform a t-test exactly as before, but splitting mice by their `diet` rather than their `genotype`.
    
<div>


<br>
<br>

[Back to ToC](#toc)

## 1.3 Statistical power calculations <a id='stats.3'></a>

Statistical **power** is the probability that a test correctly rejects the Null Hypothesis (if the alternative Hypothesis is true).

For some widely used tests, functions exist to let you automatically compute statistical power for a given effect size or sample size.

In [ ]:
from statsmodels.stats.power import TTestIndPower

mean_difference = 1
standard_dev = 1
sample_size = 10

effect_size = mean_difference/standard_dev

P = TTestIndPower()
print('power:', P.power(effect_size=effect_size, nobs1=sample_size, ratio=1, alpha=0.05))

**Calculating statistical power can help inform our experimental design**. 

For example, how many observation per sample do we need if we want to detect a difference in mean of 1 with significance level (type I error) 0.01 and statistical power 0.80:

In [ ]:
effect_size = 1
sig_threshold = 0.01
P = TTestIndPower()

powers = []
for sample_size in range(2, 50):
    powers.append(P.power(effect_size=effect_size, nobs1=sample_size, ratio=1, alpha=sig_threshold))

fig, ax = plt.subplots(figsize=(14, 7))
sns.lineplot(x=range(2, 50), y=powers, label="effect size=" + str(effect_size), ax=ax)
ax.axhline(0.8, color="r", linestyle="-")
ax.set(xlabel="sample size", ylabel="power")


# Or, directly:
print("minimum sample size:",
      P.solve_power(
          effect_size=effect_size,
          nobs1=None,
          ratio=1,
          alpha=sig_threshold,
          power=0.8
      )
)

Functions to compute power not only exists for t-test, but also for others such as Chi-square, Anova, F-test, Z-test.

See the [documentation](https://www.statsmodels.org/dev/stats.html#power-and-sample-size-calculations) for a lists and details.

<br>
<br>

[Back to ToC](#toc)

## 1.4 Multiple hypothesis testing <a id='stats.4'></a>

Recall the **definition of the p-value**: the probability of obtaining a test statistic at least as extreme as the one observed, **if the null hypothesis is true**

Thus, *even* if the p-value is, let's say, 0.04, there is still a 4% chance of obtaining such an extreme result **purely by chance**.

This is often acceptable if we only perform one test. However when **multiple tests** are performed, it was shown (by simulation), that even when there is no real effect some tests will turn out significant by chance.

> This is the definition of the $\alpha$ risk of type I error.

Of course, this has important implication for science and the relevance of our results.

![image.png](img/xkcd882.png)

> source: [xkcd](http://xkcd.com) (note: there are many relevant xkcd strips for everything related to stats/programming courses)

In [ ]:
P = np.array([1, 2, 5, 10, 50, 100])
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(P, 1 - 0.95 ** P)
ax.set(xlabel="number of tests", ylabel="probability of at least 1 test significant by chance")



We need to change perspective.
Instead of trying to limit the false positive probability for *each* test, we focus on:
* The probability of obtaining **any** false positives (family-wise error rate, **FWER**).
* The proportion of false positives among all findings (false discovery rate, **FDR**).

> Controlling the FWER is often too stringent - limit type I errors, but get lots of type II errors. 

<br>

### The Bonferroni method for controlling the FWER
* Assume we are performing $N$ tests.
* To control the FWER at (e.g.) 0.05, only call variables with p-values below $0.05/N$ significant.

<br>

### The Benjamini-Hochberg method for controlling the FDR

- Assume we are performing $N$ tests.
- Intuition: for each p-value threshold $\alpha$, we can estimate the number of false discoveries by $\alpha N$.
- Compare this to the actual number of discoveries at the threshold - $N_\alpha$.
- Choose a p-value threshold $\alpha$ such that $\alpha N/N_\alpha$ is less than a desired threshold
  (e.g. 0.05) - this threshold would give an expected FDR of 0.05.
- Note that the FDR is truly a property of a *set* - in a set of genes with FDR = 0.05, we can expect
  around 5% to be false discoveries. However, we don't know *which* ones! It could be the most significant!
- Often, we want a gene-wise measure of significance (like the p-value).
- The q-value, or adjusted p-value, of a variable is the *smallest* FDR we have to accept in order to
  call that variable significant.
- For example, if the adjusted p-value is 0.2, we have to accept that if we want to call this variable
  (and consequently, all variables with lower p-values) significant, there will be approximately 20%
  false discoveries among them.

In python :

```python
from statsmodels.stats.multitest import multipletests

# Bonferroni
rejected, fwers, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='bonferroni')

# BH procedure
rejected, fdrs, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='fdr_bh')

# Note: many other methods are available -> help(multipletests)
```

* rejected : true for hypothesis that can be rejected for the given alpha.
* fwers|fdrs : p-values corrected for multiple tests.
* alphacSidak : corrected alpha for Sidak method.
* alphacBonf : corrected alpha for Bonferroni method.

<br>

**Example:**
* Imagine we perform 10'000 t-tests on 10'000 couple of random samples all taken from the exact same distribution:
  any **significant difference in mean value among these samples** (i.e. this is what the t-test is testing)
  **will be purely by chance** (and would lead us to wrongly reject the null hypothesis).

In [ ]:
%%time
pvals = []  # A list where we will store the p-values of our t-tests.

N = 10000
mean_difference = 0 # no differences -> any detected difference is due to chance.
sample_size = 100
std=1

for i in range(N):
    t, pval_ttest = stats.ttest_ind(np.random.randn(sample_size) * std, 
                                    np.random.randn(sample_size) * std + mean_difference, equal_var=True)
    pvals.append(pval_ttest)
    
pvals = np.array(pvals)
# stats models proposes a function implementing numerous p-value correction methods
# https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html

from statsmodels.stats.multitest import multipletests

rejected, fwers, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method="bonferroni")
rejected, fdrs, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method="fdr_bh")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(pvals, bins = np.arange(0, 1.05, 0.01), ax=ax[0], label="p-value", color="xkcd:vomit")
sns.histplot(fwers, bins = np.arange(0, 1.05, 0.01), ax=ax[0], label="FWER", color="xkcd:tomato")
sns.histplot(fdrs, bins = np.arange(0, 1.05, 0.01), ax=ax[0], label="FDR", color="xkcd:lavender")
ax[0].set_yscale("log")
ax[0].legend()

ax[1].scatter(pvals, fwers, label="FWER", c="xkcd:tomato", alpha=0.5)
ax[1].scatter(pvals, fdrs, label="FDR", c="xkcd:lavender", alpha=0.5)
ax[1].set_xlabel("p-value")
ax[1].set_ylabel("corrected value")

print("Fraction of (spuriously) significant tests:")
print("p-value:", sum(pvals<0.05)/N )
print("FWER   :", sum(fwers<0.05)/N )
print("FDR    :", sum(fdrs <0.05)/N )

* Now, imagine a different scenario where 100 out of the 10'000 tests should indeed show a significant difference (because the underlying distributions from where the random numbers are drawn are different). 

In [ ]:
%%time
pvals = []

N = 10000 - 100
mean_difference = 0  # no differences -> any detected difference is due to chance.
sample_size = 100
std = 1

for i in range(N):
    t, pval_ttest = stats.ttest_ind(np.random.randn( sample_size ) * std, 
                                    np.random.randn( sample_size ) * std + mean_difference, equal_var=True)
    pvals.append(pval_ttest)

# Now we add the different ones:
mean_difference = 0.75
for i in range(100):
    t, pval_ttest = stats.ttest_ind(np.random.randn( sample_size ) * std, 
                                    np.random.randn( sample_size ) * std + mean_difference, equal_var=True)
    pvals.append(pval_ttest)
    
    
pvals = np.array(pvals)
# stats models proposes a function implementing numerous p-value correction methods
# https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html

from statsmodels.stats.multitest import multipletests

rejected, fwers, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method="bonferroni")
rejected, fdrs, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method="fdr_bh")

In [ ]:
_df = pd.DataFrame({"PV": pvals, "D": N*[1] + 100 * [0]})
sns.histplot(
    data=_df, x="PV",
    hue="D", palette="pastel",
    multiple="stack", binwidth=0.025,
    legend=False
)

In [ ]:
print('Number of significant tests:')
print('p-value:', sum(pvals<0.05))
print('FWER   :', sum(fwers<0.05))
print('FDR    :', sum(fdrs <0.05))
print()
print('Number of correctly significant tests (out of 100):')
print('p-value:', sum(pvals[-100:]<0.05) )
print('FWER   :', sum(fwers[-100:]<0.05) )
print('FDR    :', sum(fdrs[-100:] <0.05) )
print()
print('Number of spuriously significant tests (out of 9900):')
print('p-value:', sum(pvals[:-100]<0.05) )
print('FWER   :', sum(fwers[:-100]<0.05) )
print('FDR    :', sum(fdrs[:-100] <0.05) )

print("Observed False Discovery Rate:")
print('p-value: {:.3f}'.format(sum(pvals[:-100]<0.05)/sum(pvals<0.05)))
print('FWER   : {:.3f}'.format(sum(fwers[:-100]<0.05)/sum(fwers<0.05)))
print('FDR    : {:.3f}'.format(sum(fdrs[:-100] <0.05)/sum(fdrs<0.05)))

<br>
<br>

[Back to ToC](#toc)

## 1.5 Fisher's exact test and the Chi-square test <a id='stats.5'></a>

* These two tests have for object the association between 2 categorical variables.
* Their **null hypothesis** is the absence of association between the two variable.

**Fisher's exact test**, as its name entails, computes a p-value which is exact, even for very low sample sizes. However it becomes computationally complex to compute as the data set size increases or number of categories gets higher.

The **Chi-square test**, in contrast, uses an approximation of the exact p-value which is only valid when samples are big enough. However, it scales well to larger samples sizes and number of categories.


Both tests start from a **contingency table**.

Here we are going to use as example the historical [Lady tasting tea](https://en.wikipedia.org/wiki/Lady_tasting_tea).

|  | detected as milk before | detected as milk after | marginal sums |
|---|---|---|---|
| **milk before** | 3 | 1 | **4** |
| **milk after** | 1 | 3 | **4** |
| **marginal sums**  | **4** | **4** | **8** |

In our experiment, the lady was able to correctly identify 6 out of 8 cups.





In [ ]:
# Create a contingency table with the results from the tea tasting.
table = [[3,1], [1,3]]

# Fisher's exact test.
oddsratio, pvalue = stats.fisher_exact(table)
print("Fisher's exact test")
print("\todds ratio:", oddsratio)
print("\tp-value:", pvalue)

# Chi-square.
chi2, pval, df, expected = stats.chi2_contingency(table, correction=False)
print("Chi-square test")
print("\tchi2:", chi2)
print("\tp-value:", pval)

In [ ]:
expected

As you can see above, the p-values between Fisher's and the Chi-square tests are quite different.

> Note that here we use `correction=False` as by default scipy implementation uses [Yates's correction](https://en.wikipedia.org/wiki/Yates%27s_correction_for_continuity), which is useful when the numbers are low. Try the same lines with the correction to see the difference.



Let's imagine now that we have many cups and a very patient lady so that the contingency table looks like this:

|  | detected as milk before | detected as milk after | marginal sums |
|---|---|---|---|
| **milk before** | 30 | 10 | **40** |
| **milk after** | 18 | 22 | **40** |
| **marginal sums**  | **48** | **32** | **80** |



In [ ]:
table = [[30,10], [18,22]]

oddsratio , pvalue = stats.fisher_exact(table)
print("Fisher's exact test")
print('\todds ratio:', oddsratio)
print('\tp-value:', pvalue)

chi2,pval , df, expected = stats.chi2_contingency(table, correction=False)
print("Chi-square test")
print('\tchi2:', chi2)
print('\tp-value:', pval)

You can see, with a larger dataset, the p-value of the Chi-square test is closer to that of Fisher's exact test (because the approximation made by the Chi-square tests is better with larger datasets).


<br>
<br>

<div class="alert alert-block alert-success">

## Exercise 3.1

Exercises are located in the dedicated notebook `exercises_course1.ipynb`.

</div>


<br>
<br>

[Back to ToC](#toc)

## 1.6 1-way anova <a id='stats.6'></a>

The **ANOVA**, or **AN**alyse **O**f **VA**riance, stands maybe among the most used (and abused) type of statistical tests to date.

The anova is used to analyze the differences among group means in a sample. 
In particular, we are going to concentrate here on the 1-way ANOVA, which evaluates the difference in means of a numerical variable across groups formed by another (single) variable.

In this sense, it is a generalization of the t-test which is limited to 2 groups only (in fact, the 1-way anova and t-test are quivalent when there are only 2 groups).

**Anova assumptions**:
* Subpopulation distributions are normal.
* Samples have equal variances.
* Observations are independent from one another.

**Test hypothesis**: 
Given $m$ groups of mean $\bar{x}_{1...m}$, each containing $n_i$ observations (for a total of $n$):
 * **Null hypothesis**: $H_0 : \bar{x}_1 = \bar{x}_2 = ... = \bar{x}_m$
 * **Alternative hypothesis**: at least one of these means differ from the others.
 
The anova relies on the idea that if the mean varies between the different group then the overall variance of all samples should be significantly greater than the variance within each group (hence the name).

<br>

**Example:**
* In the 1880 Swiss census data: is there difference in the proportion of the number of 60+ years old depending on the main language? 

In [ ]:
# Here playing with the whole dataset is a bit silly, because we have the whole population.
# So let's imagine we only went and counted this fraction in 20 communes per language.

dfFractions = pd.read_csv("data/census1880_fractions.csv")
dfFractionsSample = dfFractions.groupby("majority language", group_keys=False).apply(lambda x: x.sample(20))
dfFractionsSample["majority language"].value_counts()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,7), sharey=True)
sns.kdeplot(data=dfFractionsSample, y="60+ y.o.", ax=axes[0])
sns.rugplot(data=dfFractionsSample, y="60+ y.o.", ax=axes[0])

sns.violinplot(data=dfFractionsSample, x="majority language", y="60+ y.o." , ax=axes[1])
_ = axes[1].set_xticks(range(4))
_ = axes[1].set_xticklabels( map(lambda x:x.partition(" ")[0] , dfFractionsSample["majority language"].unique()) , 
                            rotation = 20, rotation_mode='anchor',ha='right')
# The last line removes the "speaker" at the end of x-axis labels.

In [ ]:
# Nice trick to aggregate this data as a set of lists.
aggregated60fraction = dfFractionsSample.groupby('majority language')['60+ y.o.'].agg(list)
aggregated60fraction

In [ ]:
Fstat, pval = stats.f_oneway(*aggregated60fraction)
# equivalent to stats.f_oneway( aggregated60fraction[0] , aggregated60fraction[1] , ... )

print("automated 1-way anova / F-test:")
print("F-stat :",Fstat)
print("p-value:",pval)


In [ ]:
# With this, it can be nice to report the mean and standard deviation of each group.
dfFractionsSample.groupby('majority language')['60+ y.o.'].agg(['mean','std'])

Note that the ANOVA only tests if there is at least one significant difference between groups.

If you want to test for each differences, then **after an ANOVA**, we recommend [Tukey's honestly significant difference](https://docs.scipy.org/doc/scipy-1.8.1/reference/generated/scipy.stats.tukey_hsd.html)

In [ ]:
res = stats.tukey_hsd(*aggregated60fraction)
print(res)

In [ ]:
# The categories Ids correspond to the order in which the lists were provided:
for i, n in enumerate(aggregated60fraction.index):
    print(i,n)

In [ ]:
# We can visualize the results as a p-value heatmap.
sns.heatmap( np.log10( res.pvalue ), 
            xticklabels = aggregated60fraction.index , 
            yticklabels = aggregated60fraction.index  )

> Of course, there exists many other tests implemented in `scipy.stats` (as well as other libraries). Here is a little [cheat-sheet](https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/) to help you explore this. 

> Scipy.stats documentation is also great!

<br>
<br>

[Back to ToC](#toc)

# 2. Correlation and linear regression <a id='reg'></a>
-----------------------------------------------

## 2.1 Correlation <a id='reg.1'></a>

**Correlation** is a measure of the amount of relatedness between two measured variables. It comes in several flavours :
 * **Pearson's linear correlation** coefficient: for linear relationship only: **`stats.pearsonr(x,y)`**
 * **Spearman's rank correlation** coefficient: accepts non linear, but "monotonic only":
   **`stats.spearmanr(x,y)`**
 * **Kendall's Tau**: relies on value order relation only, and is less influenced by the scale than
   the two others: **`stats.kendalltau(x,y)`**



In [ ]:
sigma=1./5

linear=[[u,(u)/100+sigma*np.random.randn()] for u in range(10,500)]
monotonic=[[u,50*(0.8**(u/10))+sigma*np.random.randn()] for u in range(10,500)]

non_monotonic=[[u,(u)**3+3*u**2+sigma*np.random.randn()] for u in np.arange(-1,1,1./250)]

together=[linear,monotonic,non_monotonic]
fig, ax = plt.subplots(1,3,figsize=(15,5))
for i in range(3):
    x=[u[0] for u in together[i]]
    y=[u[1] for u in together[i]]
    ax[i].scatter(x,y)
    ax[i].set_title('Pearson   : {0:.3f}, \nSpearman: {1:.3f}, \nKendall   : {2:.3f}'.format(
                                    stats.pearsonr(x,y)[0],##just like that
                                    stats.spearmanr(x,y)[0],
                                    stats.kendalltau(x,y)[0]))
fig.tight_layout()    
plt.show()


[back to toc](#toc)

<br>

## 2.2 Regression <a id='reg.2'></a>

Performing regression (linear or otherwise) is possible with `scipy`, but it is not the best module for this task.

`statsmodels` offers much nicer options and statistical reports. 

<!--
Additionally, this is a great opportunity to see together how to install a library. 

1. Go to [https://www.statsmodels.org](https://www.statsmodels.org).
2. Click on the install page.
3. Find the instruction for installation with Anaconda.
4. Type the command in either a terminal (mac, linux) or in the anaconda-prompt (windows).
-->

<br>

Let's load our data using pandas:

In [ ]:
df_diabetes = pd.read_table("data/diabetes.csv", sep=",")

# Rename columns to replace spaces by underscore
df_diabetes.rename( columns = lambda x : x.replace(" ","_") , inplace=True)
df_diabetes.head()

In [ ]:
x = df_diabetes["bmi"]                  # Covariable bmi.
y = df_diabetes["disease_progression"]  # Response variable disease progression.

fig, ax = plt.subplots(figsize=(10, 10))  # Setup graphical windows.
sns.scatterplot(x=x, y=y)                 # Plot x versus y.

In [ ]:
stats.pearsonr(df_diabetes["bmi"], df_diabetes["disease_progression"])

In [ ]:
# We'll be using the statsmodel package, which computes a lot of cool metrics for you.
import statsmodels
import statsmodels.formula.api as smf

model = smf.ols("disease_progression ~ bmi" , data=df_diabetes)

## Alternative syntax: 
# import statsmodels.api as sm
# X = sm.add_constant(x) # Adding a intercept to the model.
# model = sm.OLS(y, X)   # Defining an Ordinary Least Square variable.

results = model.fit()  # Fitting the model
print(results.summary())

In [ ]:
# Diagnostic plots:
from statsmodels.graphics.gofplots import qqplot

fig, axes = plt.subplots(1, 2, figsize = (15, 6))

# Plotting residuals (error of the model) vs. fitted values (prediction fo the model)
# helps determine homosedascticity and sphericity of errors:
# the points should show about the same spread all along the x axis, and be centered around 0.
axes[0].scatter( results.fittedvalues , results.resid )
axes[0].axhline(0.0, color="grey")
axes[0].set_xlabel("fitted values")
axes[0].set_ylabel("residuals")

# The QQplot (quantile-quantile plot) is  great plot to assess normality of the
# model's residuals. Basically points should stay close to the diagonal line if
# they follow something close to a normal distribution.
qqplot(results.resid, ax=axes[1], line="s") 
plt.show()


In [ ]:
# Plotting the fit.
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# We obtain the predicted values for our model, as well as their 95% intervals.
prstd, iv_l, iv_u = wls_prediction_std(results) 

fig, ax = plt.subplots(figsize=(10,10))

#ax.plot(x, y, 'o', label="data")
sns.scatterplot(x=x,y=y) # plot x versus y
ax.plot(x, results.fittedvalues, "r", label="OLS regression result")
ax.plot(x, iv_u, color="orange",linestyle="--" , label="95% fit interval")
ax.plot(x, iv_l, color="orange",linestyle="--")
ax.legend(loc="best", fontsize=10)
#plt.yscale('log')

This is a simple ordinary least square linear regression with a single covariable, 
but stats models is able to handle much more complex models such as [GLMs](https://www.statsmodels.org/stable/glm.html) , [time series analysis](https://www.statsmodels.org/stable/tsa.html), [survival analysis](https://www.statsmodels.org/stable/duration.html), ...

<br>
<br>

<div class="alert alert-block alert-success">

## Exercise 3.2 - Free-form exercise

Exercises are located in the dedicated notebook `exercises_course1.ipynb`.

</div>


[Back to ToC](#toc)